Importing libraries

In [ ]:
import tensorflow as tf

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
import mlflow

In [ ]:
from mlflow.models.signature import infer_signature

In [ ]:
import numpy as np

In [ ]:
tfds.list_builders()

In [ ]:
data, meta_data = tfds.load(name='mnist',	
                            as_supervised=True,
                            with_info=True,
                            shuffle_files=True)

In [ ]:
train_data = data['train']
test_data = data['test']

In [ ]:
class_names = meta_data.features['label'].names
class_names

In [ ]:
train_data = train_data.shuffle(buffer_size=1000)
sample = train_data.take(1)
for image, label in sample:
  sample_image = image
  sample_label = label

import matplotlib.pyplot as plt
plt.imshow(sample_image)
plt.title(sample_label.numpy())

In [ ]:
sample_image.shape

In [ ]:
def preprocessing_image(image, label, imagesize=28):
  image = tf.image.resize(images=image,
                          size=[imagesize, imagesize],
                          )
  return tf.cast(image, dtype=tf.float32)/255., label

In [ ]:
train_data = train_data.map(map_func=preprocessing_image, num_parallel_calls=tf.data.AUTOTUNE)
train_data = train_data.shuffle(buffer_size=1000)
train_data = train_data.batch(batch_size=32)
train_data = train_data.prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
test_data = test_data.map(map_func=preprocessing_image, num_parallel_calls=tf.data.AUTOTUNE).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

SEQUENTAIL MODEL

In [ ]:
from keras.models import Sequential
from keras.layers import Convolution2D, GlobalAveragePooling1D, Dense, Flatten

In [ ]:
model = Sequential([
        Convolution2D(filters=10,
        kernel_size=3,
        strides=1,
        padding='valid',
        activation='relu',
        input_shape=(28, 28, 1)), # input layer (specify input shape)
  Convolution2D(10, 3, activation='relu'),
  Convolution2D(10, 3, activation='relu'),
  Flatten(),
  Dense(10, activation='sigmoid') # output layer (specify output shape)
])

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", # Use sparse_categorical_crossentropy when labels are *not* one-hot
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
experiment_id = mlflow.create_experiment(name='mnist_data_mlflow_tracking')

In [ ]:
mlflow.set_experiment(experiment_name='mnist_data_mlflow_tracking')

In [ ]:
artifact_path = 'mnist_tf_model_mlflow'

In [ ]:
mlflow.tensorflow.autolog(log_models=False)

In [ ]:
with mlflow.start_run(run_name='testing_tensorflow_auto_logging', nested=True) as tf_run:
    history1 = model.fit(train_data,
                    epochs=5,
                    steps_per_epoch=len(train_data)//32,
                    validation_data=test_data,
                    validation_steps=int(0.15 * len(test_data)))
    # Infer model signature
    sample = train_data.take(1)
    for image, label in sample:
        sample_image =  image.numpy()
    signature = infer_signature(model_input=sample_image, model_output=model.predict(sample_image))


    mlflow.tensorflow.log_model(model=model, artifact_path=artifact_path, signature=signature)

In [ ]:
dir(tf_run)

In [ ]:
tf_run.info.run_id, tf_run.info.run_name

In [ ]:
model_uri = f'runs:/{tf_run.info.run_id}/mnist_tf_model_mlflow'

In [ ]:
load_model = mlflow.pyfunc.load_model(model_uri=model_uri)

In [ ]:
for image, lable in test_data.take(1):
    test_sample = image.numpy()
    lable = lable.numpy()

In [ ]:
lable

In [ ]:
test_sample.shape

In [ ]:
predict_logged_model = load_model.predict(test_sample)

In [ ]:
predict_logged_model[0].argmax()

Saving Model Keras Native format

In [ ]:
mlflow.set_experiment(experiment_name='mnist_data_mlflow_tracking')

In [ ]:
mlflow.tensorflow.autolog(log_models=False)

In [ ]:
with mlflow.start_run(run_name='testing_tensorflow_auto_logging_keras_model', nested=True) as tf_run:
    history1 = model.fit(train_data,
                    epochs=5,
                    steps_per_epoch=len(train_data)//32,
                    validation_data=test_data,
                    validation_steps=int(0.15 * len(test_data)))
    # Infer model signature
    sample = train_data.take(1)
    for image, label in sample:
        sample_image =  image.numpy()
    signature = infer_signature(model_input=sample_image, model_output=model.predict(sample_image))


    mlflow.keras.log_model(model=model, artifact_path=artifact_path, signature=signature)

Registering and deploying the model

In [ ]:
model_name = 'mnist_keras_model'

In [ ]:
tf_run.info.run_id

In [ ]:
run_df = mlflow.search_runs()

In [ ]:
run_df['run_id'][0]

In [ ]:
model_bath = f'runs:/389191edf6e04ac283addb17c5f6c69c/mnist_tf_model_mlflow'

In [ ]:
model_version = mlflow.register_model(model_uri=model_bath, name=model_name)

invoking client

In [ ]:
from mlflow.tracking import MlflowClient

In [ ]:
client = MlflowClient()

In [ ]:
client.get_model_version_stages(name=model_name, version=model_version.version)

In [ ]:
client.transition_model_version_stage(name=model_name, version=model_version.version, stage='Production')

In [ ]:
#predict using the model in production stage
model_path = f'models:/{model_name}/Production'
load_production_model = mlflow.keras.load_model(model_uri=model_path)

In [ ]:
predict_production_model = load_production_model.predict(test_sample)

In [ ]:
predict_production_model[0].argmax()

In [ ]:
client.update_registered_model(name=model_name, description='Keras Model')

In [ ]:
#mlflow.register_model(model_uri='runs:/039e0293de764b1f845814bba7890d6f/mnist_tf_model_mlflow', name=model_name)

In [ ]:
#run_df

In [ ]:
#client.transition_model_version_stage(name=model_name, version=1, stage='Archived')

In [ ]:
#client.transition_model_version_stage(name=model_name, version=2, stage='Archived')

Deploying MOdels in Azure Endpoints

In [ ]:
from azureml.core import Workspace

In [ ]:
ws = Workspace.create(name='mlflow_model_deployment', subscription_id='43257558-8ed1-49ab-901d-5615fd372599', resource_group='mlflow_testing', location='eastus', create_resource_group=False)

In [ ]:
ws = Workspace.get(name='mlflow_model_deployment', subscription_id='43257558-8ed1-49ab-901d-5615fd372599', resource_group='mlflow_testing', location='eastus')

In [ ]:
from azureml.core.model import Model

In [ ]:
path = os.path.abspath('D:/learning_desk/MLOPS/mlruns/472875462688718728/389191edf6e04ac283addb17c5f6c69c/artifacts/mnist_tf_model_mlflow')

In [ ]:
model = Model.register(workspace=ws, model_path=path, model_name='mnist_keras_model')

In [ ]:
import json
from mlflow.deployments import get_deploy_client

# Create the deployment configuration.
# If no deployment configuration is provided, 
# then the deployment will happen using ACI (Azure Container Instance).
deploy_config = {"computeType": "aci"}

# Write the deployment configuration into a file.
deployment_config_path = "deployment_config.json"

with open(deployment_config_path, "w") as outfile:
    outfile.write(json.dumps(deploy_config))


In [ ]:
from mlflow.deployments import get_deploy_client

In [ ]:
client = get_deploy_client(target_uri='azureml://eastus.api.azureml.ms/mlflow/v1.0/subscriptions/43257558-8ed1-49ab-901d-5615fd372599/resourceGroups/mlflow_testing/providers/Microsoft.MachineLearningServices/workspaces/mlflow_model_deployment')

config = {"deploy-config-file": deployment_config_path}
model_name = 'mnist_keras_model'
model_version = 1

In [ ]:
mlflow.set_tracking_uri('azureml://eastus.api.azureml.ms/mlflow/v1.0/subscriptions/43257558-8ed1-49ab-901d-5615fd372599/resourceGroups/mlflow_testing/providers/Microsoft.MachineLearningServices/workspaces/mlflow_model_deployment')

In [ ]:
client.create_deployment(name='test_deployment', model_uri=f"models:/{model_name}/{model_version}",
                        config=config)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(5, 5))
plt.plot(history1.history['loss'], label='Traing Loss')
plt.plot(history1.history['val_loss'], label='Validation Loss')
plt.legend()

plt.figure(figsize=(5, 5))
plt.plot(history1.history['accuracy'], label='Traing Accuracy')
plt.plot(history1.history['val_accuracy'], label='Validation Accuracy')
plt.legend()

In [ ]:
def load_and_prep_image(filename, img_shape=28):
  """
  Reads an image from filename, turns it into a tensor
  and reshapes it to (img_shape, img_shape, colour_channel).
  """
  # Read in target file (an image)
  img = tf.io.read_file(filename)

  # Decode the read file into a tensor & ensure 3 colour channels
  # (our model is trained on images with 3 colour channels and sometimes images have 4 colour channels)
  img = tf.image.decode_image(img, channels=1)

  # Resize the image (to the same size our model was trained on)
  img = tf.image.resize(img, size = [img_shape, img_shape])

  # Rescale the image (get all values between 0 and 1)
  img = img/255.
  return img

In [ ]:
loaded_img = load_and_prep_image(filename='three.jpg')

In [ ]:
plt.imshow(loaded_img)

In [ ]:
loaded_img = tf.expand_dims(loaded_img, axis=0)
loaded_img.shape

In [ ]:
predict_1 = model.predict(loaded_img)
predict_1

In [ ]:
predict_1.argmax()

In [ ]:
class_names[predict_1.argmax()]

In [ ]:
tf.__version__